### Goal
<Br>
A/B tests play a huge role in website optimization. Analyzing A/B tests data is a very important
data scientist responsibility. Especially, data scientists have to make sure that results are
reliable, trustworthy, and conclusions can be drawn.
<Br>
Furthermore, companies often run tens, if not hundreds, of A/B tests at the same time. Manually
analyzing all of them would require lot of time and people. Therefore, it is common practice to
look at the typical A/B test analysis steps and try to automate as much as possible. This frees
up time for the data scientists to work on more high level topics.
<Br>
In this challenge, you will have to analyze results from an A/B test. Also, you will be asked to
design an algorithm to automate some steps.
    
### Challenge Description
<Br>
Company XYZ is a worldwide e-commerce site with localized versions of the site.
<Br>
A data scientist at XYZ noticed that Spain-based users have a much higher conversion rate than
any other Spanish-speaking country. She therefore went and talked to the international team in
charge of Spain And LatAm to see if they had any ideas about why that was happening.
<Br>
Spain and LatAm country manager suggested that one reason could be translation. All Spanishspeaking
countries had the same translation of the site which was written by a Spaniard. They
agreed to try a test where each country would have its one translation written by a local. That is,
Argentinian users would see a translation written by an Argentinian, Mexican users by a Mexican
and so on. Obviously, nothing would change for users from Spain.
<Br>
After they run the test however, they are really surprised cause the test is negative. I.e., it
appears that the non-localized translation was doing better!
<Br>
You are asked to:
<Br>
- Confirm that the test is actually negative. That is, it appears that the old version of the
site with just one translation across Spain and LatAm performs better
- Explain why that might be happening. Are the localized translations really worse?
- If you identified what was wrong, design an algorithm that would return FALSE if the
same problem is happening in the future and TRUE if everything is good and the results
can be trusted.


#### "test_table" - general information about the test results
#### Columns:
<Br>
- user_id : the id of the user. Unique by user. Can be joined to user id in the other table.
For each user, we just check whether conversion happens the first time they land on the
site since the test started.
- date : when they came to the site for the first time since the test started
- source : marketing channel: Ads, SEO, Direct . Direct means everything except for ads
and SEO. Such as directly typing site URL on the browser, downloading the app w/o
coming from SEO or Ads, referral friend, etc.
- device : device used by the user. It can be mobile or web
- browser_language : in browser or app settings, the language chosen by the user. It can
be EN, ES, Other (Other means any language except for English and Spanish)
- ads_channel : if marketing channel is ads, this is the site where the ad was displayed. It
can be: Google, Facebook, Bing, Yahoo ,Other. If the user didn't come via an ad, this
field is NA
- browser : user browser. It can be: IE, Chrome, Android_App, FireFox, Iphone_App,
Safari, Opera
- conversion : whether the user converted (1) or not (0). This is our label. A test is
considered successful if it increases the proportion of users who convert.
- test : users are randomly split into test (1) and control (0). Test users see the new
translation and control the old one. For Spain-based users, this is obviously always 0
since there is no change there.
    
#### "user_table" - some information about the user
#### Columns:
<Br>
- user_id : the id of the user. It can be joined to user id in the other table
- sex : user sex: Male or Female
- age : user age (self-reported)
- country : user country based on ip address

In [108]:
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np
import scipy.stats
import math as mt
from scipy.stats import norm

In [11]:
df_test = pd.read_csv('https://raw.githubusercontent.com/SSSSYYYY/Take-home/master/2.ab_test.csv')


In [13]:
df = df.drop('Unnamed: 0',axis = 1)
df.head()
## df size : (453321, 9)

,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test
0,315281,2015-12-03,Direct,Web,ES,NaN,IE,1,0
1,497851,2015-12-04,Ads,Web,ES,Google,IE,0,1
2,848402,2015-12-04,Ads,Web,ES,Facebook,Chrome,0,0
3,290051,2015-12-03,Ads,Mobile,Other,Facebook,Android_App,0,1
4,548435,2015-11-30,Ads,Web,ES,Google,FireFox,0,1


### explortory 

In [30]:
3# Check missing values
pd.isnull(df).any()

user_id             False
date                False
source              False
device              False
browser_language    False
ads_channel          True
browser             False
conversion          False
test                False
dtype: bool

In [31]:
df[df.ads_channel.isnull()]

## 271444 (60%) user with no ads channel recorded

,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test
0,315281,2015-12-03,Direct,Web,ES,NaN,IE,1,0
5,540675,2015-12-03,Direct,Mobile,ES,NaN,Android_App,0,1
6,863394,2015-12-04,SEO,Mobile,Other,NaN,Android_App,0,0
7,527287,2015-12-03,Direct,Web,EN,NaN,Chrome,0,0
8,261625,2015-12-04,Direct,Mobile,ES,NaN,Android_App,0,1
11,73335,2015-12-03,SEO,Web,ES,NaN,IE,0,1
12,744811,2015-12-01,SEO,Mobile,ES,NaN,Android_App,1,0
13,234023,2015-12-03,SEO,Web,ES,NaN,Chrome,0,0
15,103879,2015-11-30,Direct,Mobile,ES,NaN,Iphone_App,0,0
16,409295,2015-12-02,Direct,Web,ES,NaN,FireFox,0,1


In [32]:
df = df.fillna('None')

## since we cannot remove all the records with missing values, we'll put a place holder 

In [34]:
## check all the columns

for column in df.columns:
    uniques = sorted(df[column].unique())
    print('{0:20s} {1:5d} \t {2:}'.format(column, len(uniques), df[column].dtype) ,'   ',uniques[:7])
    
## 453321 the IDs are all unique 
## the experienment was 5 days from 2015-11-30 (Mon) to 2015-12-04 (Fri)


user_id              453321 	 int64     [1, 2, 3, 5, 8, 11, 12]
date                     5 	 object     ['2015-11-30', '2015-12-01', '2015-12-02', '2015-12-03', '2015-12-04']
source                   3 	 object     ['Ads', 'Direct', 'SEO']
device                   2 	 object     ['Mobile', 'Web']
browser_language         3 	 object     ['EN', 'ES', 'Other']
ads_channel              6 	 object     ['Bing', 'Facebook', 'Google', 'None', 'Other', 'Yahoo']
browser                  7 	 object     ['Android_App', 'Chrome', 'FireFox', 'IE', 'Iphone_App', 'Opera', 'Safari']
conversion               2 	 int64     [0, 1]
test                     2 	 int64     [0, 1]


In [88]:
## test and control

df.groupby('test').user_id.count()

test
0    237338
1    215983
Name: user_id, dtype: int64

#### Sanity check

In [165]:
def sanity_check(test,control,p, alpha):
    SE = np.sqrt((p*(1-p))/(test+control))
    m = round(norm.ppf(1-(alpha/2))*SE,4)
    
    pool_p = round(float(test)/(test+control),4)
    if (pool_p > p-m) & (pool_p < p+m):
        return print ('The true percentage',pool_p, 'is inside of CI (',p-m,',', p+m,'), the two numbers are comparable.')
                      
    else:
                      return print ('The true percentage',pool_p, 'is out of CI (',p-m,',', p+m,'), the two numbers are not comparable.')
                      


In [112]:
## this is to make sure the filters are the same for both control and test, and if they are comparable
## we also have to check the for the invariance metric

true_p = 0.5
alpha=0.05
t_cont = 237338
t_exp = 215983
SE = np.sqrt((true_p*(1-true_p))/(t_cont+t_exp))
m = round(norm.ppf(1-(alpha/2))*SE,4)
CI = (true_p-m,true_p+m)

pool_p = float(t_cont)/(t_exp+t_cont)
pool_p

print ("The confidence interval is between",true_p-m,"and",true_p+m,"; Is",round((t_cont)/(t_exp+t_cont),4),"inside this range?")

The confidence interval is between 0.4985 and 0.5015 ; Is 0.5236 inside this range?


##### the test and control is not randomized

In [47]:
## source

print(df[df.test == 0].groupby('source').conversion.agg(['count','mean']))
print(df[df.test == 1].groupby('source').conversion.agg(['count','mean']))

        count      mean
source                 
Ads     95348  0.055481
Direct  47740  0.056221
SEO     94250  0.054345
        count      mean
source                 
Ads     86529  0.043546
Direct  43094  0.042489
SEO     86360  0.043770


In [49]:
## device

print(df[df.test == 0].groupby('device').conversion.agg(['count','mean']))
print(df[df.test == 1].groupby('device').conversion.agg(['count','mean']))

         count      mean
device                  
Mobile  105400  0.055693
Web     131938  0.054768
         count      mean
device                  
Mobile   96356  0.044003
Web     119627  0.042959


In [50]:
## browser_language

print(df[df.test == 0].groupby('browser_language').conversion.agg(['count','mean']))
print(df[df.test == 1].groupby('browser_language').conversion.agg(['count','mean']))


                   count      mean
browser_language                  
EN                 33003  0.053631
ES                197773  0.055452
Other               6562  0.054709
                   count      mean
browser_language                  
EN                 30134  0.042610
ES                179774  0.043532
Other               6075  0.044280


In [51]:
## ads_channel  

print(df[df.test == 0].groupby('ads_channel').conversion.agg(['count','mean']))
print(df[df.test == 1].groupby('ads_channel').conversion.agg(['count','mean']))

              count      mean
ads_channel                  
Bing           7174  0.055060
Facebook      35789  0.056777
Google        35771  0.054737
None         141990  0.054976
Other          2215  0.050113
Yahoo         14399  0.055143
              count      mean
ads_channel                  
Bing           6515  0.042057
Facebook      32636  0.044184
Google        32409  0.043013
None         129454  0.043344
Other          1933  0.037765
Yahoo         13036  0.044876


In [52]:
## browser

print(df[df.test == 0].groupby('browser').conversion.agg(['count','mean']))
print(df[df.test == 1].groupby('browser').conversion.agg(['count','mean']))


             count      mean
browser                     
Android_App  80944  0.056026
Chrome       53127  0.054605
FireFox      21388  0.057041
IE           32471  0.054603
Iphone_App   24456  0.054588
Opera         3203  0.057134
Safari       21749  0.052830
             count      mean
browser                     
Android_App  74191  0.043469
Chrome       48802  0.043851
FireFox      19378  0.041697
IE           29244  0.043052
Iphone_App   22165  0.045793
Opera         2887  0.049879
Safari       19316  0.040795


In [81]:
## date

print(df[df.test == 0].groupby('date').conversion.agg(['count','mean']))
print(df[df.test == 1].groupby('date').conversion.agg(['count','mean']))


            count      mean
date                       
2015-11-30  37142  0.057374
2015-12-01  37116  0.053616
2015-12-02  37100  0.055364
2015-12-03  52073  0.056094
2015-12-04  73907  0.054122
            count      mean
date                       
2015-11-30  33883  0.043886
2015-12-01  33875  0.041387
2015-12-02  33549  0.044234
2015-12-03  47420  0.043884
2015-12-04  67256  0.043491


#### Sign test

In [ ]:
# Number of "successes": 5
# Number of trials (or subjects) per experiment: 5
# Sign test. If the probability of "success" in each trial or subject is 0.500, then:
# The one-tail P value is 0.0313
# This is the chance of observing 5 or more successes in 5 trials.

## this is statistically significant, that means they are differrnt

#### A/B Testing

In [68]:
control = df[df['test'] == 0 ].conversion
test = df[df['test'] == 1 ].conversion

ttest_ind(control, test,equal_var=False)

## the result is significint 
## the conversionrate has droped after the change of local translation

Ttest_indResult(statistic=18.312271285740266, pvalue=7.030776744312677e-75)

In [14]:
df_user =  pd.read_csv('https://raw.githubusercontent.com/SSSSYYYY/Take-home/master/2.user_table.csv')
df_user.head()

,user_id,sex,age,country
0,765821,M,20,Mexico
1,343561,F,27,Nicaragua
2,118744,M,23,Colombia
3,987753,F,27,Venezuela
4,554597,F,20,Spain


In [73]:
## join the user table and explore more
df_all = pd.merge(df,df_user, how = 'left', left_on = ['user_id'], right_on = ['user_id'])

In [74]:
df_all.head()

,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test,sex,age,country
0,315281,2015-12-03,Direct,Web,ES,None,IE,1,0,M,32.0,Spain
1,497851,2015-12-04,Ads,Web,ES,Google,IE,0,1,M,21.0,Mexico
2,848402,2015-12-04,Ads,Web,ES,Facebook,Chrome,0,0,M,34.0,Spain
3,290051,2015-12-03,Ads,Mobile,Other,Facebook,Android_App,0,1,F,22.0,Mexico
4,548435,2015-11-30,Ads,Web,ES,Google,FireFox,0,1,M,19.0,Mexico


In [79]:
df_all.groupby(['browser_language','country']).user_id.count()

browser_language  country    
EN                Argentina        6419
                  Bolivia          1556
                  Chile            2733
                  Colombia         7700
                  Costa Rica        777
                  Ecuador          2185
                  El Salvador      1163
                  Guatemala        2096
                  Honduras         1206
                  Mexico          17789
                  Nicaragua         917
                  Panama            543
                  Paraguay         1048
                  Peru             4732
                  Spain            7196
                  Uruguay           567
                  Venezuela        4452
ES                Argentina       38997
                  Bolivia          9254
                  Chile           16433
                  Colombia        44850
                  Costa Rica       4378
                  Ecuador         13262
                  El Salvador      6775
          

In [78]:
df_all.groupby(['country','browser_language']).user_id.count()

country      browser_language
Argentina    EN                    6419
             ES                   38997
             Other                 1317
Bolivia      EN                    1556
             ES                    9254
             Other                  314
Chile        EN                    2733
             ES                   16433
             Other                  571
Colombia     EN                    7700
             ES                   44850
             Other                 1510
Costa Rica   EN                     777
             ES                    4378
             Other                  154
Ecuador      EN                    2185
             ES                   13262
             Other                  448
El Salvador  EN                    1163
             ES                    6775
             Other                  237
Guatemala    EN                    2096
             ES                   12610
             Other                  419
Honduras  

In [179]:
print(df_all[(df_all['test'] == 0) & (df_all.country != 'Spain') & (df_all.browser_language == 'ES')].conversion.mean())
print(df_all[(df_all['test'] == 1) & (df_all.country != 'Spain') & (df_all.browser_language == 'ES')].conversion.mean())

0.048659031087714306
0.04353243516860058


In [80]:
control_2 = df_all[(df_all['test'] == 0) & (df_all.country != 'Spain') & (df_all.browser_language == 'ES')].conversion
test_2 = df_all[(df_all['test'] == 1) & (df_all.country != 'Spain') & (df_all.browser_language == 'ES')].conversion

ttest_ind(control_2, test_2,equal_var=False)

Ttest_indResult(statistic=7.035374097759072, pvalue=1.9912166469879827e-12)

In [83]:
## date

print(df_all[df_all.test == 0].groupby('country').conversion.agg(['count','mean']))
print(df_all[df_all.test == 1].groupby('country').conversion.agg(['count','mean']))

             count      mean
country                     
Argentina     9356  0.015071
Bolivia       5550  0.049369
Chile         9853  0.048107
Colombia     27088  0.052089
Costa Rica    2660  0.052256
Ecuador       8036  0.049154
El Salvador   4108  0.053554
Guatemala     7622  0.050643
Honduras      4361  0.050906
Mexico       64209  0.049495
Nicaragua     3419  0.052647
Panama        1966  0.046796
Paraguay      3650  0.048493
Peru         16869  0.049914
Spain        51782  0.079719
Uruguay        415  0.012048
Venezuela    16149  0.050344
             count      mean
country                     
Argentina    37377  0.013725
Bolivia       5574  0.047901
Chile         9884  0.051295
Colombia     26972  0.050571
Costa Rica    2649  0.054738
Ecuador       7859  0.048988
El Salvador   4067  0.047947
Guatemala     7503  0.048647
Honduras      4207  0.047540
Mexico       64275  0.051186
Nicaragua     3304  0.054177
Panama        1985  0.049370
Paraguay      3697  0.049229
Peru         1

In [113]:
## subset the experienment by contires we can find there's a huge differnce between the contries
## we also see there's discripancy in terms of # of subject between the test and control (Argentina, Uruguay)

subset_country = df_all.groupby(['country','test']).conversion.agg(['count','mean'])

In [141]:
for i in subset_country.index.levels[0] :
    print(i)

Argentina
Bolivia
Chile
Colombia
Costa Rica
Ecuador
El Salvador
Guatemala
Honduras
Mexico
Nicaragua
Panama
Paraguay
Peru
Spain
Uruguay
Venezuela


In [167]:
subset_country

count      mean Sanity_check
country     test                              
Argentina   0      9356  0.015071             
            1     37377  0.013725             
Bolivia     0      5550  0.049369             
            1      5574  0.047901             
Chile       0      9853  0.048107             
            1      9884  0.051295             
Colombia    0     27088  0.052089             
            1     26972  0.050571             
Costa Rica  0      2660  0.052256             
            1      2649  0.054738             
Ecuador     0      8036  0.049154             
            1      7859  0.048988             
El Salvador 0      4108  0.053554             
            1      4067  0.047947             
Guatemala   0      7622  0.050643             
            1      7503  0.048647             
Honduras    0      4361  0.050906             
            1      4207  0.047540             
Mexico      0     64209  0.049495             
            1     64275  0.051186             
Nicaragua   0      3419  0.052647             
            1      3304  0.054177             
Panama      0      1966  0.046796             
            1      1985  0.049370             
Paraguay    0      3650  0.048493             
            1      3697  0.049229             
Peru        0     16869  0.049914             
            1     16797  0.050604             
Spain       0     51782  0.079719             
Uruguay     0       415  0.012048             
            1      3719  0.012907             
Venezuela   0     16149  0.050344             
            1     15905  0.048978

In [ ]:
## design sanity tests for the subsets and the p values for the subsets

In [161]:
sanity_check(subset_country.loc['Ecuador',1]['count'],subset_country.loc['Ecuador',0]['count'],0.05,0.5)

The true percentage 0.4944 is out of CI ( 0.0488 , 0.0512 ), the two numbers are not comparable.


In [154]:
subset_country['Sanity_check'] =''

In [160]:
subset_country.loc['Ecuador',]

,count,mean,Sanity_check
test,,,
0,8036,0.049154,
1,7859,0.048988,


In [ ]:
lambda : 

In [178]:
for country in subset_country.index.levels[0] :
    
    if country == 'Spain':
        continue
    print(country)
    sanity_check(subset_country.loc[country,1]['count'],subset_country.loc[country,0]['count'],0.5,0.05)
    
    

Argentina
The true percentage 0.7998 is out of CI ( 0.4955 , 0.5045 ), the two numbers are not comparable.
Bolivia
The true percentage 0.5011 is inside of CI ( 0.4907 , 0.5093 ), the two numbers are comparable.
Chile
The true percentage 0.5008 is inside of CI ( 0.493 , 0.507 ), the two numbers are comparable.
Colombia
The true percentage 0.4989 is inside of CI ( 0.4958 , 0.5042 ), the two numbers are comparable.
Costa Rica
The true percentage 0.499 is inside of CI ( 0.4866 , 0.5134 ), the two numbers are comparable.
Ecuador
The true percentage 0.4944 is inside of CI ( 0.4922 , 0.5078 ), the two numbers are comparable.
El Salvador
The true percentage 0.4975 is inside of CI ( 0.4892 , 0.5108 ), the two numbers are comparable.
Guatemala
The true percentage 0.4961 is inside of CI ( 0.492 , 0.508 ), the two numbers are comparable.
Honduras
The true percentage 0.491 is inside of CI ( 0.4894 , 0.5106 ), the two numbers are comparable.
Mexico
The true percentage 0.5003 is inside of CI ( 0.4973

In [176]:
print(check)

None


In [177]:
check = sanity_check(subset_country.loc[country,1]['count'],subset_country.loc[country,0]['count'],0.5,0.05)

The true percentage 0.4962 is inside of CI ( 0.4945 , 0.5055 ), the two numbers are comparable.


#### Conclusion

1. the test result is not reliable
2. the test and control are not randomly choosen
3. subset the data with country, we can see different country has different ratios
4. the result (local translator make it worse), is not true, there's conpounding effect in the subsets
5. we can either redo the experiement with ramdom sampling or analyse the result by countries